In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!nvidia-smi

# 1. Clone yolov7 


In [ ]:
# Clone source code YOLOv7 về thư mục OD_train   
%cd /content/drive/MyDrive/OD_train
!git clone https://github.com/augmentedstartups/yolov7.git

#2. Download required packages

In [ ]:
# Cài đặt các thư viện cần thiết để train YOLOv7
%cd /content/drive/MyDrive/OD_train/yolov7
!pip install -r requirements.txt

#3. Download pretrained model

In [ ]:
# Tải weight pretrain 
%cd /content/drive/MyDrive/OD_train/yolov7
!mkdir pretrain
%cd pretrain
!rm yolov7.pt
!wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7.pt

#4. Prepare dataset

### Clone soucre Data

In [ ]:
# Clone source Data về thư mục OD_train   
%cd /content/drive/MyDrive/OD_train
!git clone https://github.com/dotrannhattuong/Dataset_OD_SelfDrivingCar

### Check data

In [ ]:
import glob
txt_files_glob = glob.glob('/content/drive/MyDrive/OD_train/Dataset_OD_SelfDrivingCar/yolo_data_detect/*/*/*.txt')
images_files_glob = glob.glob('/content/drive/MyDrive/OD_train/Dataset_OD_SelfDrivingCar/yolo_data_detect/*/*images/*')
len(txt_files_glob), len(images_files_glob)

(7568, 7568)

### Split to train/val/test folder

In [3]:
from sklearn.model_selection import train_test_split
import glob
import os
from shutil import copyfile

In [4]:
import glob

txt_files_glob = glob.glob('/content/drive/MyDrive/OD_train/Dataset_OD_SelfDrivingCar/yolo_data_detect/*/*/*.txt')

for txt in txt_files_glob:
  img_name = txt.split('/')[-1]
  img_name = img_name.split('.')
  img_name = '.'.join(img_name[:-1])
  copyfile(txt, os.path.join("/content/drive/MyDrive/OD_train/Dataset_OD_SelfDrivingCar/data_new_noSplit", img_name + '.txt'))


In [5]:
# Split Dataset
dataset_labels_path = '/content/drive/MyDrive/OD_train/Dataset_OD_SelfDrivingCar/yolo_data_detect'

image_list = glob.glob(f'{dataset_labels_path}/*/*images/*')
train_test_list, val_list = train_test_split(image_list, test_size=0.2, random_state=42)
train_list, test_list = train_test_split(train_test_list, test_size=0.125, random_state=42)

print('total =',len(image_list))
print('train :',len(train_list))
print('val  :',len(val_list))
print('test  :',len(test_list))

total = 7568
train : 5297
val  : 1514
test  : 757


In [6]:
 # train split
for filename in train_list:
    img_name = filename.split('/')[-1]
    img_name = img_name.split('.')
    img_name = '.'.join(img_name[:-1])
    copyfile(os.path.join("/content/drive/MyDrive/OD_train/Dataset_OD_SelfDrivingCar/data_new_noSplit", img_name + '.txt'), os.path.join("/content/drive/MyDrive/OD_train/Dataset_OD_SelfDrivingCar/data_new/train/labels", img_name + '.txt'))
    copyfile(filename, os.path.join("/content/drive/MyDrive/OD_train/Dataset_OD_SelfDrivingCar/data_new/train/images", img_name + '.png'))

  # val split
for filename in val_list:
    img_name = filename.split('/')[-1]
    img_name = img_name.split('.')
    img_name = '.'.join(img_name[:-1])
    copyfile(os.path.join("/content/drive/MyDrive/OD_train/Dataset_OD_SelfDrivingCar/data_new_noSplit", img_name + '.txt'), os.path.join("/content/drive/MyDrive/OD_train/Dataset_OD_SelfDrivingCar/data_new/val/labels", img_name + '.txt'))
    copyfile(filename, os.path.join("/content/drive/MyDrive/OD_train/Dataset_OD_SelfDrivingCar/data_new/val/images", img_name + '.png'))

  # test split
for filename in test_list:
    img_name = filename.split('/')[-1]
    img_name = img_name.split('.')
    img_name = '.'.join(img_name[:-1])
    copyfile(os.path.join("/content/drive/MyDrive/OD_train/Dataset_OD_SelfDrivingCar/data_new_noSplit", img_name + '.txt'), os.path.join("/content/drive/MyDrive/OD_train/Dataset_OD_SelfDrivingCar/data_new/test/labels", img_name + '.txt'))
    copyfile(filename, os.path.join("/content/drive/MyDrive/OD_train/Dataset_OD_SelfDrivingCar/data_new/test/images", img_name + '.png'))

### Visualize Data

In [ ]:
from sklearn.model_selection import train_test_split
import glob
import os
from shutil import copyfile

In [ ]:
import altair as alt
import pandas as pd
import numpy as np

In [ ]:
lst = []
for file in glob.glob('/content/drive/MyDrive/OD_train/Dataset_OD_SelfDrivingCar/yolo_data_detect/*/*/*.txt'):
  df = pd.read_csv(file, sep=" ", header=None, names=['class', 'x', 'y', 'w', 'h'])
  lst.append(df)
signs = pd.concat(lst)
print(signs)

In [ ]:
data = pd.DataFrame()
data['counts'] = signs['class'].value_counts()
data.index = ['turn_right', 'straight', 'no_turn_left', 'no_turn_right', 'no_straight', 'car', 'unknown', 'turn_left']
data['title'] = data.index
print(data)

In [ ]:
alt.Chart(data).mark_bar().encode(
    x=alt.X('counts', axis=alt.Axis(title='Counts')),
    y=alt.Y('title',
        sort=alt.EncodingSortField(field='counts', order='ascending', op='sum'),
        axis=alt.Axis(title='Title')
    ),
    color=alt.Color('title:N')
)

#5. Prepare custom .yaml file


In [ ]:
# Khai báo 1 file yaml để YOLOv7 biết:
# - Đường dẫn đến thư mục train, test (nếu có, nếu không thì dùng luôn đường dẫn đến train)
# - Số lượng class qua biến nc (number of class)
# - Tên của các class
%cd /content/drive/MyDrive/OD_train/yolov7
!rm data/mydataset.yaml # nếu có
!echo 'train: /content/drive/MyDrive/OD_train/Dataset_OD_SelfDrivingCar/data_new/train' >> data/mydataset.yaml
!echo 'val: /content/drive/MyDrive/OD_train/Dataset_OD_SelfDrivingCar/data_new/val' >> data/mydataset.yaml
!echo 'nc: 9' >> data/mydataset.yaml
!echo "names: ['no', 'turn_right', 'straight', 'no_turn_left', 'no_turn_right', 'no_straight', 'car', 'unknown', 'turn_left']" >> data/mydataset.yaml

# 6. Train

### First Training

In [ ]:
# Train model YOLOv7 với dữ liệu OD
%cd /content/drive/MyDrive/OD_train/yolov7
!python train.py --batch 16 --cfg cfg/training/yolov7.yaml --epochs 300 --data data/mydataset.yaml --weights 'pretrain/yolov7.pt' --hyp data/hyp.scratch.p5.yaml

/content/drive/MyDrive/OD_train/yolov7
YOLOR 🚀 v0.1-104-g941b94c torch 1.13.0+cu116 CUDA:0 (Tesla T4, 15109.75MB)

Namespace(adam=False, artifact_alias='latest', batch_size=16, bbox_interval=-1, bucket='', cache_images=False, cfg='cfg/training/yolov7.yaml', data='data/mydataset.yaml', device='', entity=None, epochs=300, evolve=False, exist_ok=False, freeze=[0], global_rank=-1, hyp='data/hyp.scratch.p5.yaml', image_weights=False, img_size=[640, 640], label_smoothing=0.0, linear_lr=False, local_rank=-1, multi_scale=False, name='exp', noautoanchor=False, nosave=False, notest=False, project='runs/train', quad=False, rect=False, resume=False, save_dir='runs/train/exp', save_period=-1, single_cls=False, sync_bn=False, total_batch_size=16, upload_dataset=False, weights='pretrain/yolov7.pt', workers=8, world_size=1)
tensorboard: Start with 'tensorboard --logdir runs/train', view at http://localhost:6006/
hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0

### Resume 

In [ ]:
%cd /content/drive/MyDrive/CD_train/yolov7

!python train.py --resume /content/drive/MyDrive/UTE2023/train/exp/weights/last.pt --cache

#7. Inference

In [ ]:
#Nhận diện thử với weights vừa train
%cd /content/drive/MyDrive/OD_train/yolov7
!python detect.py --weights /content/drive/MyDrive/OD_train/exp2/weights/best.pt --source /content/drive/MyDrive/OD_train/test

In [ ]:
# Xem ảnh đã nhận diện
from IPython.display import Image, display
display(Image(filename="/content/drive/MyDrive/OD_train/yolov7/runs/detect/exp"))